### 不使用框架，手工构建RAG，基于DeepSeek和Ollama生成模型

加载环境变量

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
# 配置 DeepSeek API 密钥
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
if not DEEPSEEK_API_KEY:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

1. 加载与读取文档

In [3]:
text = '';
with open('data/黑神话wiki.txt', 'rb') as f:
    text = f.read().decode('utf-8')

2. 分割文档
- 为了更精确地实现基于向量的语义检索，我们不能简单地嵌入文档的内容，而是需要分割：将文档分割成最合适的大小的多个知识块（Chunk）后做向量化

In [4]:
import re

def split_text_by_sentences(source_text: str, sentences_per_chunk: int = 3, overlap: int = 1) ->list[str]:
    """
    简单地把文档分割成多个知识块，每个知识块都包含指定数量的句子

    参数:
    source_text (str): 要分割的文本
    sentences_per_chunk (int): 每个知识块包含的句子数量，默认为3
    overlap (int): 相邻知识块之间的句子重叠数量，默认为1
    
    返回:
        list[str]: 分割后的知识块列表
    """
    # 参数检查
    if not isinstance(source_text, str):
        raise ValueError("输入必须是字符串类型")
    if not source_text.strip():
        raise ValueError("输入文本不能为空")
    if sentences_per_chunk < 1:
        raise ValueError("每个知识块至少包含1个句子")
    if overlap < 0 or overlap >= sentences_per_chunk:
        raise ValueError("重叠句子数必须大于等于0且小于每个知识块的句子数")
    
    # print(f"文本长度: {len(source_text)}")
    # print(f"文本内容前100个字符: {source_text[:100]}")
    
    # 简单化处理， 用正则表达式来分割句子
    sentences = re.split(r'(?<=[。！？.!?])\s*', source_text)
    # print(f"分割后的句子数量: {len(sentences)}")
    sentences = [s.strip() for s in sentences if s.strip()]

    # 如果句子数量不足，直接返回整个文本
    if len(sentences) <= sentences_per_chunk:
        return [source_text]
    
    chunks = []
    step = sentences_per_chunk - overlap
    
    # 生成知识块
    for i in range(0, len(sentences), step):
        chunk = sentences[i:i + sentences_per_chunk]
        if len(chunk) < sentences_per_chunk and i > 0:
            # 如果最后一个知识块句子不足，且不是第一个知识块，则跳过
            continue
        chunks.append(' '.join(chunk))
    
    return chunks

3. 设置嵌入模型

In [5]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
chunks = split_text_by_sentences(source_text=text, sentences_per_chunk=8, overlap=2)
embeddings = embedding_model.encode(chunks)
print(f"文档向量维度: {embeddings.shape}")

No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


文档向量维度: (21, 384)


4. 创建向量存储

In [6]:
# 创建向量存储
import chromadb
from chromadb.config import Settings

# 初始化 Chroma 客户端，设置持久化存储
client = chromadb.PersistentClient(path="./chroma_db")

# 创建或获取集合
collection = client.get_or_create_collection(
    name="black_myth_wiki",  # 集合名称
    metadata={"hnsw:space": "cosine"}  # 使用余弦相似度
)

# 准备数据
ids = [f"doc_{i}" for i in range(len(chunks))]
metadatas = [{"source": "黑神话wiki"} for _ in range(len(chunks))]

# 将数据添加到集合中
collection.add(
    embeddings=embeddings.tolist(),
    documents=chunks,
    ids=ids,
    metadatas=metadatas
)

print(f"成功存储了 {len(chunks)} 个文档块到向量数据库")

成功存储了 21 个文档块到向量数据库


4. 执行相似度检索

In [7]:
question = "黑神话悟空的游戏有什么特点?"

# 将问题转换为向量
question_embedding = embedding_model.encode(question)
# 在向量数据库中搜索最相似的文档
results = collection.query(
    query_embeddings=[question_embedding.tolist()],
    n_results=3  # 返回最相似的3个文档
)
# 打印检索结果
print("\n检索到的最相关文档片段：")
for i, (doc, score) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n片段 {i+1} (相似度得分: {1-score:.4f}):")
    print(doc)


检索到的最相关文档片段：

片段 1 (相似度得分: 0.5359):
天命人与二郎神及四大天王在梅山展开激战[46]. 天命人杀死四大天王，并击败杨戬，杨戬的“第三眼”释放出孙悟空的第六件根器“意见欲”交给天命人，并分享他的领悟——原来之前的战斗实际上是让孙悟空通过肉身的死亡，从而让孙悟空找到摆脱束缚的道路. [47][48]

结局
天命人和猪八戒回到花果山，到达天真顶的石卵后，进入孙悟空执念所化的幻境“石中境”. 老猴子带领天命人和猪八戒渡过识海，老猴子提到孙悟空的第六件根器“意见欲”已经消逝，因为“意根”是每个生命独特的本质，注定会在生命终结时消散，所以孙悟空永远不会回来. 这次旅程的真正目的，是要天命人成为替身，继承孙悟空的根本和名号. 为了完成轮回，西游重续，天命人必须击败大圣残躯才能完成天命. [47]

最后，天命人与代表孙悟空“执念”的大圣残躯战斗. 大圣残躯被击败后逐渐消散，紧箍儿掉入水中.

片段 2 (相似度得分: 0.5281):
[75][76]

发行
2023年1月，官方发布兔年短片，公布游戏的发布日期为2024年夏[77]. 开发团队致力于将游戏发布于PC与主流游戏主机平台[78]. 游戏将以买断制的形式发行，并可能有下载包. 2023年8月20日，在杭州举办线下试玩会，体验内容包括多个首领挑战以及独立的关卡片段[79]. 12月官方发布发售日预告，公布该游戏将登陆 PC、PS5 和 Xbox Series X/S 平台，也不排除支持可流畅运行的云游戏平台. [7]

2024年2月，国家新闻出版署发布了当月国产网络游戏审批信息以及版号变更信息，包括《黑神话：悟空》在内的共111款游戏过审，标志着《黑神话：悟空》可以在中国大陆公开发售. [80]5月，在WeGame游戏之夜上，官方公布游戏将同步上线WeGame平台. [81]

企业联合创始人刘卓解释游戏主角派生品的设计过程.

片段 3 (相似度得分: 0.5191):
[47]

最后，天命人与代表孙悟空“执念”的大圣残躯战斗. 大圣残躯被击败后逐渐消散，紧箍儿掉入水中. 在决战之前如果天命人没有从二郎神那里获得孙悟空的第六件根器，孙悟空将不会复活，老猴子会将紧箍儿戴在天命人头上，天命人沦为天庭执行意志的新工具；但如果在决战之前天命人已经获得孙悟空的第六件根器，则六根齐聚，孙

5. 构建提示词

In [8]:
prompt = f"""
    请基于以下的上下文回答问题，如果上下文中不包含足够的回答问题的信息，请回答'我暂时无法回答该问题题'，不要编造。
    上下文：
    {doc}
    我的问题是：{question}
"""

6. 使用DeepSeek生成答案

In [9]:
from openai import OpenAI
client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1"
)

response = client.chat.completions.create(
    model="deepseek-chat",  
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=1024
)
print(f"\n生成的答案: {response.choices[0].message.content}")


生成的答案: 根据上下文，《黑神话：悟空》的特点包括：  
1. **剧情分支**：游戏结局受玩家选择影响（如是否集齐第六件根器决定孙悟空是否复活及是否被天庭束缚）。  
2. **战斗与探索**：预告片展示了主角在黑风山探索环境并与敌人战斗的实机内容。  
3. **技术引擎**：基于虚幻引擎4开发，画面表现力强。  
4. **开发背景**：初期团队规模小（7人），通过首段实机演示吸引人才，团队后续扩增至约30人。  

上下文未提及其他具体玩法或系统设计，因此无法进一步补充。


7. 使用Ollama本地大模型生成答案

In [10]:
from ollama import chat

response = chat(
    model='qwen2.5:3b',  
    messages=[{
        "role": "user",
        "content": prompt
    }],
)
print(f"\n生成的答案: {response.message.content}")


生成的答案: 《黑神话：悟空》游戏有以下特点：
- 该游戏由游戏科学于2017年开始开发。
- 2020年8月20日公布首段实机演示视频，展示了基于虚幻引擎4开发的游戏内容。 
- 游戏中主角在黑风山探索环境，并与各种敌人战斗。
- 发布预告片后，在YouTube和Bilibili平台上获得了大量观看次数。
